# Problem: Categorize Bank Accounts by Salary Range

#### Table: Accounts

| Column Name | Type |
|-------------|------|
| account_id  | int  |
| income      | int  |

- `account_id` is the primary key (column with unique values) for this table.
- Each row contains information about the monthly income for one bank account.

### Task
Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:
- `Low Salary`: All the salaries strictly less than $20,000.
- `Average Salary`: All the salaries in the inclusive range [$20,000, $50,000].
- `High Salary`: All the salaries strictly greater than $50,000.

The result table must:
- Contain all three categories.
- Return `0` if there are no accounts in a category.
- Be returned in any order.

### Example

### Input

#### Accounts table:

| account_id | income |
|------------|--------|
| 3          | 108939 |
| 2          | 12747  |
| 8          | 87709  |
| 6          | 91796  |

### Output

| category       | accounts_count |
|----------------|----------------|
| Low Salary     | 1              |
| Average Salary | 0              |
| High Salary    | 3              |

### Explanation

- `Low Salary`: Only Account 2 has an income of $12,747, which is strictly less than $20,000.
- `Average Salary`: No accounts fall within the income range [$20,000, $50,000].
- `High Salary`: Accounts 3, 6, and 8 have incomes greater than $50,000.


In [3]:
import sqlite3
import pandas as pd

In [4]:
# Connect to the database (or create it if it doesn't exist) with a busy_timeout of 5 seconds
conn = sqlite3.connect('example.db')
conn.execute('PRAGMA busy_timeout = 5000;')  # Timeout in milliseconds (5 seconds)

# Create a cursor object
cursor = conn.cursor()

In [7]:
# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Accounts (
    account_id int,
    income int
)
''')

In [8]:
# Truncate the table (delete all existing rows)
cursor.execute('DELETE FROM Accounts')

# Insert data
data_to_insert = [
    (3, 108939),
    (2, 12747),
    (7, 87709),
    (6, 91796),
]

cursor.executemany('''
INSERT INTO Accounts (account_id, income)
VALUES (?, ?)
''', data_to_insert)

In [9]:
# Simple solution with UNION ALL

query = ('''
SELECT "Low Salary" AS category,
    COUNT(account_id) AS accounts_count
FROM Accounts
WHERE income < 20000
UNION ALL
SELECT "Average Salary" AS category,
    COUNT(account_id) AS accounts_count
FROM Accounts
WHERE income BETWEEN 20000 AND 50000
UNION ALL
SELECT "High Salary" AS category,
    COUNT(account_id) AS accounts_count
FROM Accounts
WHERE income > 50000;
''')

pd.read_sql_query(query, conn)

,category,accounts_count
0,Low Salary,1
1,Average Salary,0
2,High Salary,3


In [ ]:
# Create a temporary table SalaryCategories
cursor.execute('''
CREATE TEMPORARY TABLE IF NOT EXISTS SalaryCategories (
    category VARCHAR(20),
    accounts_count int
)
''')

In [ ]:
# Truncate the table (delete all existing rows)
cursor.execute('DELETE FROM SalaryCategories')

# Fill a temporary table SalaryCategories
data_to_insert = [
    ("Low Salary", 0),
    ("Average Salary", 0),
    ("High Salary", 0),
]

cursor.executemany('''
INSERT INTO SalaryCategories
VALUES (?, ?)
''', data_to_insert)

# Sort accounts by income
cursor.execute('''
UPDATE SalaryCategories
SET accounts_count = (
    SELECT COUNT(*)
    FROM Accounts
    WHERE
        (category = "Low Salary" AND income < 20000) OR
        (category = "Average Salary" AND income BETWEEN 20000 AND 50000) OR
        (category = "High Salary" AND income > 50000)
)
''')

pd.read_sql_query('SELECT * FROM SalaryCategories', conn)

,category,accounts_count
0,Low Salary,1
1,Average Salary,0
2,High Salary,3


In [ ]:
# Commit the connection
conn.commit()

# Close the connection
conn.close()